# CART

In [1]:
import numpy as np
import pandas as pd 
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
from sklearn.preprocessing import scale 
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score,roc_curve
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

from warnings import filterwarnings
filterwarnings('ignore')

In [8]:
diabet=pd.read_csv("diabetes.csv")
df=diabet.copy()
df=df.dropna()
y=df["Outcome"]
X=df.drop(["Outcome"],axis=1)
X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=0.30, random_state=42) 

# model & karar inceleme

In [3]:
from sklearn.tree import DecisionTreeClassifier

In [4]:
cart=DecisionTreeClassifier()
cart.fit(X_train,y_train)

DecisionTreeClassifier()

In [6]:
#burada cart yapısını bilyorsun if else lere göre ağaç kurar işte bu if else leri bi göreyim dersen
from skompiler import skompile
print(skompile(cart.predict).to("python/code")) #değişken çok derinlik sayısnı kısıltlamadık bu yüzden karışık bir yapı çıktı

((((((0 if x[6] <= 0.671999990940094 else 1 if x[6] <= 0.6974999904632568 else
    0) if x[5] <= 31.40000057220459 else ((0 if x[3] <= 40.5 else 1) if x[1
    ] <= 111.5 else ((1 if x[1] <= 123.0 else 0) if x[2] <= 65.0 else 0) if
    x[2] <= 72.0 else 1) if x[4] <= 9.0 else (0 if x[6] <= 
    0.6395000219345093 else 1 if x[6] <= 0.6759999990463257 else 0) if x[0] <=
    4.5 else 1 if x[7] <= 26.5 else 0) if x[5] <= 49.10000038146973 else 1) if
    x[1] <= 127.5 else 1 if x[2] <= 56.0 else (0 if x[7] <= 27.5 else 1 if 
    x[1] <= 138.0 else 0) if x[5] <= 30.300000190734863 else 1 if x[5] <= 
    32.000000953674316 else ((0 if x[5] <= 33.75 else 1) if x[0] <= 0.5 else
    1 if x[5] <= 32.45000076293945 else 0) if x[2] <= 85.0 else 1) if x[7] <=
    28.5 else (1 if x[7] <= 29.5 else (0 if x[1] <= 133.0 else (1 if x[6] <=
    0.3685000091791153 else 0) if x[1] <= 135.0 else 0) if x[2] <= 94.0 else
    1 if x[0] <= 9.0 else 0) if x[5] <= 26.949999809265137 else (1 if x[1] <=
    28.5 else

In [7]:
#daha sade bir yapı göremek istersen
#değişken sayısnı azaltır
X_train=X_train["Pregnancies"]
X_train=pd.DataFrame(X_train)


#ve derinliği kısıltlarız
cart_sade=DecisionTreeClassifier(max_depth=2)
cart_sade.fit(X_train,y_train)
print(skompile(cart_sade.predict).to("python/code"))

((0 if x[0] <= 2.5 else 0) if x[0] <= 6.5 else 1 if x[0] <= 13.5 else 1)



# Tuning

In [13]:
cart=DecisionTreeClassifier()
params={"max_depth":range(1,10), #cart'da da pek çok paremetre var sadece ikş tanesş ğzerinde işlme yapıcaz burada. yine aşırı öğrenme ile mücadeel ediliyor burada
        "min_samples_split":(list(range(2,50)))}#gördüğün üzere hem direkt range kullanabilirsin hemdi liste dönüştürebilirsin farl etmez :)

In [14]:
cart_cv=GridSearchCV(cart,params,cv=10,n_jobs=-1,verbose=2)
cart_cv.fit(X_train,y_train)

Fitting 10 folds for each of 432 candidates, totalling 4320 fits
[CV] END ...................max_depth=1, min_samples_split=2; total time=   0.0s
[CV] END ...................max_depth=1, min_samples_split=2; total time=   0.0s
[CV] END ...................max_depth=1, min_samples_split=2; total time=   0.0s
[CV] END ...................max_depth=1, min_samples_split=3; total time=   0.0s
[CV] END ...................max_depth=1, min_samples_split=3; total time=   0.0s
[CV] END ...................max_depth=1, min_samples_split=3; total time=   0.0s
[CV] END ...................max_depth=1, min_samples_split=3; total time=   0.0s
[CV] END ...................max_depth=1, min_samples_split=3; total time=   0.0s
[CV] END ...................max_depth=1, min_samples_split=3; total time=   0.0s
[CV] END ...................max_depth=1, min_samples_split=3; total time=   0.0s
[CV] END ...................max_depth=1, min_samples_split=3; total time=   0.0s
[CV] END ...................max_depth=1, min

GridSearchCV(cv=10, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'max_depth': range(1, 10),
                         'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11,
                                               12, 13, 14, 15, 16, 17, 18, 19,
                                               20, 21, 22, 23, 24, 25, 26, 27,
                                               28, 29, 30, 31, ...]},
             verbose=2)

In [15]:
cart_cv.best_params_

{'max_depth': 5, 'min_samples_split': 19}

In [18]:
cart_tuned=DecisionTreeClassifier(max_depth=5,min_samples_split=19)
cart_tuned.fit(X_train, y_train)
cart_tuned.score(X_test,y_test)

0.7532467532467533

In [19]:
#hocam anladığım mean squared error'un küçük
#accuracy scor'un bğyğk omasını istiyoruz :)))